In [9]:
# https://www.kaggle.com/joparga3/in-depth-skewed-data-classif-93-recall-acc-now/notebook

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


**Why do I use this kernel?**

I have a deeply interest in find out if a transaction is a fraud or not. 
About the Data set: 
*     it contains transactions made by credit cards in September 2013 by european cardholders.
*     It's highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.
*     It contains only numerical input variables which are the result of a PCA transformation (confidential data).

In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
# Read data
df = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')

In [12]:
# Check the columns and general structure
df.head()

## As we can see, the data is confidential, we don't know the column names.

In [13]:
# Search missing values
df.isnull().sum().max()

## No missing values

# See if the data is balanced or unbalanced

In [14]:
from matplotlib.legend_handler import HandlerBase
from matplotlib.text import Text
class TextHandler(HandlerBase):
    def create_artists(self, legend, tup ,xdescent, ydescent, width, height, fontsize,trans):
        tx = Text(width/2.,height/2,tup[0], fontsize=fontsize,
                  ha="center", va="center", color=tup[1], fontweight="bold")
        return [tx]
    
ax = sns.countplot(x="Class", data=df)
plt.title("Frauds vs Not. Frauds")
plt.xlabel("Target")
plt.ylabel("Count")
sns.set(rc = {'figure.figsize':(8,8)})
#Add value counts to each bar
for p in ax.patches:
   ax.annotate('{:.1f}'.format(p.get_height()), (p.get_x()+0.25, p.get_height()+0.01))

# Add right legend
handltext = ["0", "1"]
labels = ["Not Fraud", "Fraud"]

t = ax.get_xticklabels()
labeldic = dict(zip(handltext, labels))
labels = [labeldic[h.get_text()]  for h in t]
handles = [(h.get_text(),c.get_fc()) for h,c in zip(t,ax.patches)]

ax.legend(handles, labels, handler_map={tuple : TextHandler()}) 



In [15]:
# The classes are heavily skewed we need to solve this issue later.
print('No Frauds', round(df['Class'].value_counts()[0]/len(df) * 100,2), '% of the dataset')
print('Frauds', round(df['Class'].value_counts()[1]/len(df) * 100,2), '% of the dataset')

**Distributions:** By seeing the distributions we can have an idea how skewed are these features, we can also see further distributions of the other features. 

In [16]:
fig, ax = plt.subplots(1, 2, figsize=(18,4))

amount_val = df['Amount'].values
time_val = df['Time'].values

sns.distplot(amount_val, ax=ax[0], color='r')
ax[0].set_title('Distribution of Transaction Amount', fontsize=14)
ax[0].set_xlim([min(amount_val), max(amount_val)])

sns.distplot(time_val, ax=ax[1], color='b')
ax[1].set_title('Distribution of Transaction Time', fontsize=14)
ax[1].set_xlim([min(time_val), max(time_val)])

# Unbalance Data

## What can we do?

* Collect more data.
* Use a correct metric (don't use accuracy):
    * Use the confusio nmatrix to calculate Precision, Recall
    * F1score (weighted average of precision recall)
    * Use Kappa - which is a classification accuracy normalized by the imbalance of the classes in the data
    * ROC curves - calculates sensitivity/specificity ratio.
* Resampling the dataset (process the data to have an approximate 50-50 ratio).
    * OVER-sampling, adding copies of the under-represented class (better when you have little data).
    * UNDER-sampling, deletes instances from the over-represented class (better when he have lot's of data).
